In [114]:
!pwd

/opt/ml/level1_bookratingprediction_recsys-level1-recsys-02/wonjun


In [115]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss

In [124]:
users = pd.read_csv('../main_code/data/users.csv')
books = pd.read_csv("../main_code/data/books.csv", encoding='utf-8')
train = pd.read_csv('../main_code/data/train_ratings.csv')
test = pd.read_csv('../main_code/data/test_ratings.csv')
sub = pd.read_csv('../main_code/data/sample_submission.csv')

In [19]:
users # 68092 x 3

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0
...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0
68088,278621,"victoria, delaware, canada",74.0
68089,278636,"irvington, alabama, usa",NaN
68090,278659,"vancouver, washington, usa",33.0


In [20]:
books # 149570 x 10

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['Humor'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Nature'],A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Fiction'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [125]:
train # 306795 x 3

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7


In [22]:
test # 예측해야함 다 0으로 되어있음

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


In [23]:
sub # test랑 똑같음

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


In [117]:
def text_preprocessing_func(text : str ) -> str :
    """
    깨진 문자를 변환하는 함수
    """
    text = text.replace('Ã?Â©','e') # 원래는 é인데 걍 e로 메움
    text = text.replace('Ã©', 'e')
    text = text.replace('Ã?Â?','e') # 원래는 é인데 걍 e로 메움
    text = text.lower()
    return text

# train에서 2개만 평가한 유저의 평가 데이터를 삭제

In [274]:
train = pd.read_csv('../main_code/data/train_ratings.csv')

In [275]:
train

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7


In [276]:
user_count = train.groupby('user_id').count()['isbn'].to_dict() # user_id 항목별로 묶을 건데, isbn의 수를 세자 즉, 한 유저가 평가한 isbn 수?
for i in range(len(train)):
    train.at[i, 'count'] = user_count[train['user_id'][i]] # at은 row이름, column 이름으로 해당 데이터에 접근, train에 count라는 값이 생김
train = train[train['count']>2].reset_index(drop=True)
train = train.drop("count", axis=1)
train # 272178 x 3 그니까 306795에서 3만개가 줄었음 1개만 평가한 놈들 삭제

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,210926,0002005018,9
3,219008,0002005018,7
4,263325,0002005018,5
...,...,...,...
256433,278843,0743525493,7
256434,278851,067161746X,6
256435,278851,0884159221,7
256436,278851,0912333022,7


In [ ]:
train.to_csv('./data/train_pppp.csv', index=False) # train 내보내기

# users 데이터 가공

In [312]:
users = pd.read_csv('../main_code/data/users.csv')

In [35]:
#users = users.drop('age', axis=1)
users # age 삭제

,user_id,location
0,8,"timmins, ontario, canada"
1,11400,"ottawa, ontario, canada"
2,11676,"n/a, n/a, n/a"
3,67544,"toronto, ontario, canada"
4,85526,"victoria, british columbia, canada"
...,...,...
68087,278376,"danville, pennsylvania, usa"
68088,278621,"victoria, delaware, canada"
68089,278636,"irvington, alabama, usa"
68090,278659,"vancouver, washington, usa"


#### users 나이를 nan이면 평균으로 채워보자

In [313]:
def age_map(x: int) -> int:
    x = int(x)
    if x < 20:
        return 1
    elif x >= 20 and x < 30:
        return 2
    elif x >= 30 and x < 40:
        return 3
    elif x >= 40 and x < 50:
        return 4
    elif x >= 50 and x < 60:
        return 5
    else:
        return 6

In [ ]:
#users['age'] = users['age'].fillna(int(users['age'].mean()))
#users['age'] = users['age'].apply(age_map)

In [279]:
users # 아직 age 살아있음

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0
...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0
68088,278621,"victoria, delaware, canada",74.0
68089,278636,"irvington, alabama, usa",NaN
68090,278659,"vancouver, washington, usa",33.0


In [314]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') #특문 삭제

/tmp/ipykernel_28851/1223241613.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') #특문 삭제


In [315]:
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

In [190]:
users

,user_id,location,age,location_city,location_state,location_country
0,8,"timmins,ontario,canada",NaN,timmins,ontario,canada
1,11400,"ottawa,ontario,canada",49.0,ottawa,ontario,canada
2,11676,"na,na,na",NaN,NaN,NaN,NaN
3,67544,"toronto,ontario,canada",30.0,toronto,ontario,canada
4,85526,"victoria,britishcolumbia,canada",36.0,victoria,britishcolumbia,canada
...,...,...,...,...,...,...
68087,278376,"danville,pennsylvania,usa",54.0,danville,pennsylvania,usa
68088,278621,"victoria,delaware,canada",74.0,victoria,delaware,canada
68089,278636,"irvington,alabama,usa",NaN,irvington,alabama,usa
68090,278659,"vancouver,washington,usa",33.0,vancouver,washington,usa


In [316]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values # 나라만 비어 있는 애들의 도시 목록

In [317]:
modify_location

array(['ottawa', 'seattle', 'albuquerque', ..., 'sammamish', 'calgary',
       'bolligen'], dtype=object)

In [318]:
location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

In [319]:
location_list # modify_location에 맞는 나라 찾아주기 위한 리스트

['ottawa,ontario,canada',
 'seattle,washington,usa',
 'albuquerque,newmexico,usa',
 'humble,texas,usa',
 'aloha,oregon,usa',
 'pearland,texas,usa',
 'sarasota,florida,usa',
 'westspringfield,massachusetts,usa',
 'westlinn,oregon,usa',
 'northfortmyers,florida,usa',
 'coconutgrove,florida,usa',
 'mercerisland,washington,usa',
 'mercerisland,washington,usa',
 'desoto,missouri,usa',
 'vigo,pontevedra,spain',
 'carmichael,california,usa',
 'chicago,illinois,usa',
 'kualalumpur,kualalumpur,malaysia',
 'tucson,arizona,usa',
 'binghamton,newyork,usa',
 'yarmouth,novascotia,canada',
 'pattersonlakes,victoria,australia',
 'tucson,arizona,usa',
 'manchester,england,unitedkingdom',
 'cotati,california,usa',
 'tualatin,oregon,usa',
 'sanfrancisco,california,usa',
 'chesterfield,missouri,usa',
 'brooklyn,newyork,usa',
 'oxford,england,unitedkingdom',
 'anchorage,alaska,usa',
 'chicago,illinois,usa',
 'molalla,oregon,usa',
 'cincinnati,ohio,usa',
 'sahuarita,arizona,usa',
 'fredericton,newbrunswick,

In [320]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1] # 주 채워주기
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2] # 나라 채워주기

In [321]:
users.isnull().sum() # city 결측 122 state 결측 1132 country 결측 271

user_id                 0
location                0
age                 27833
location_city         122
location_state       1132
location_country      271
dtype: int64

In [288]:
#users['location_country'] = users['location_country'].fillna('anycountry') # 차 있지 않은 나라는 anycountry로 채워주자

In [289]:
#users['location_state'] = users['location_state'].fillna('anystate') # 차 있지 않은 state는 anystate로 채워주자

In [290]:
users.isnull().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state          0
location_country        0
dtype: int64

In [325]:
users = users.dropna(subset=['location_state', 'location_city', 'location_country']).reset_index(drop=True)
users = users.drop('location', axis=1) # 그럼에도 사는 곳 정보가 없는 애들은 삭제 그래서, 68092 -> 66803 2000명 삭제

In [326]:
users.isnull().sum()

user_id                 0
age                 27340
location_city           0
location_state          0
location_country        0
dtype: int64

In [327]:
country_fix_dict = {'usa': {'oklahoma','districtofcolumbia', 'connecticut', 'worcester', 'aroostook', 'texas',  'kern', 'orangeco', 'unitedstatesofamerica', 'fortbend', 'alachua', 'massachusetts', 'arizona', 'austin', 'hawaii', 'ohio', 'camden', 'arkansas', 'minnesota', 'losestadosunidosdenorteamerica', 'us', 'usanow', 'northcarolina', 'maine', 'colorado','oklahoma', 'alabama', 'anystate', 'districtofcolumbia', 'unitedstaes', 'pender', 'newhampshire', 'unitedstates', 'missouri', 'idaho', 'ca', 'newyork','tennessee', 'stthomasi', 'dc', 'washington', 'illinois', 'california', 'michigan', 'iowa', 'maryland', 'newjersey', 'vanwert', 'oregon'},
                    'uk': {'alderney', 'wales',  'aberdeenshire', 'bermuda', 'nottinghamshire', 'scotland', 'usacurrentlylivinginengland', 'england', 'countycork', 'alderney', 'cambridgeshire', 'middlesex', 'northyorkshire', 'westyorkshire', 'cocarlow', 'sthelena'},
                    'japan': {'okinawa'},
                    'southkorea': {'seoul'},
                    'canada': {'ontario', 'alberta', 'novascotia', 'newfoundland', 'newbrunswick', 'britishcolumbia'},
                    'miyanma': {'burma'},
                    'newzealand': {'auckland', 'nz', 'otago'},
                    'spain': {'andalucia','pontevedra', 'gipuzkoa', 'lleida', 'catalunyaspain', 'galiza', 'espaa'},
                    'germany': {'niedersachsen', 'deutschland'},
                    'brazil': {'disritofederal'},
                    'switzerland': {'lasuisse'},
                    'italy': {'veneziagiulia', 'ferrara', 'italia'},
                    'australia': {'nsw', 'queensland', 'newsouthwales'},
                    'belgium': {'labelgique', 'bergued'},
                    'uruguay': {'urugua'},
                    'panama': {'republicofpanama'}
                   }
country_del_list = ['c', 'space', 'universe', 'unknown', 'quit', 'tdzimi', 'universe', 'tn', 'unknown', 'space', 'c', 'franciscomorazan', 'petrolwarnation', 'ineurope', 'hereandthere', 'faraway'] 

In [329]:
del_idx = []
for idx, row in enumerate(users['location_country']):
    for key, value in country_fix_dict.items():
        if row in value:
            users.at[idx, 'location_country'] = key
    if row in country_del_list:
        del_idx.append(idx)
        
users = users.drop(del_idx, axis=0).reset_index(drop=True) # 제대로 나라 수정 및 이상한 나라는 삭제 66803 -> 66790 명

In [294]:
#for i in del_idx:
    #users.at[i, 'location_country'] = 'anycountry'

In [330]:
users

,user_id,age,location_city,location_state,location_country
0,8,NaN,timmins,ontario,canada
1,11400,49.0,ottawa,ontario,canada
2,67544,30.0,toronto,ontario,canada
3,85526,36.0,victoria,britishcolumbia,canada
4,96054,29.0,ottawa,ontario,canada
...,...,...,...,...,...
66785,278376,54.0,danville,california,usa
66786,278621,74.0,victoria,delaware,canada
66787,278636,NaN,irvington,alabama,usa
66788,278659,33.0,vancouver,britishcolumbia,canada


In [331]:
#users = users.drop('location_state', axis=1) # 주 정보 삭제
users = users.drop('location_city', axis=1) # 도시 정보 삭제

In [332]:
users # id, state, country만 가짐 (state 삭제 age 추가)

,user_id,age,location_state,location_country
0,8,NaN,ontario,canada
1,11400,49.0,ontario,canada
2,67544,30.0,ontario,canada
3,85526,36.0,britishcolumbia,canada
4,96054,29.0,ontario,canada
...,...,...,...,...
66785,278376,54.0,california,usa
66786,278621,74.0,delaware,canada
66787,278636,NaN,alabama,usa
66788,278659,33.0,britishcolumbia,canada


In [333]:
meanage = users['age'].mean()
meanage

36.11287951746161

In [334]:
countrymeanage = users.groupby('location_country').mean()['age'].to_dict() # 나이 정리

/tmp/ipykernel_28851/467492034.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  countrymeanage = users.groupby('location_country').mean()['age'].to_dict() # 나이 정리


In [335]:
for i in range(len(users)):
    users.at[i, 'cma'] = countrymeanage[users['location_country'][i]] # users에 나라별 평균 나이 column 생김

In [336]:
for i in range(len(users)): # 나이가 비어 있다면 나라별 평균 나이로 채움, 근데 anycountry는 전체 평균 나이로 채움
    if pd.isna(users['age'][i]):
        if users.at[i, 'location_country'] == 'anycountry':
            users.at[i, 'age'] = meanage
        else:
            users.at[i, 'age'] = users.at[i, 'cma']

In [302]:
for i in range(len(users)): # 그래도 비어 있는 애들도 평균나이로 채움
    if pd.isna(users['age'][i]):
        users.at[i, 'age'] = meanage

In [337]:
pd.isna(users['age']).value_counts() # 비어있는 애들 없음

False    66766
True        24
Name: age, dtype: int64

In [339]:
users = users.dropna(axis=0)  #나이 빈 애들은 삭제

In [340]:
users

,user_id,age,location_state,location_country,cma
0,8,35.693353,ontario,canada,35.693353
1,11400,49.000000,ontario,canada,35.693353
2,67544,30.000000,ontario,canada,35.693353
3,85526,36.000000,britishcolumbia,canada,35.693353
4,96054,29.000000,ontario,canada,35.693353
...,...,...,...,...,...
66785,278376,54.000000,california,usa,37.842036
66786,278621,74.000000,delaware,canada,35.693353
66787,278636,37.842036,alabama,usa,37.842036
66788,278659,33.000000,britishcolumbia,canada,35.693353


In [341]:
users['age'] = users['age'].astype(int)

/tmp/ipykernel_28851/3692784595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['age'] = users['age'].astype(int)


In [342]:
users

,user_id,age,location_state,location_country,cma
0,8,35,ontario,canada,35.693353
1,11400,49,ontario,canada,35.693353
2,67544,30,ontario,canada,35.693353
3,85526,36,britishcolumbia,canada,35.693353
4,96054,29,ontario,canada,35.693353
...,...,...,...,...,...
66785,278376,54,california,usa,37.842036
66786,278621,74,delaware,canada,35.693353
66787,278636,37,alabama,usa,37.842036
66788,278659,33,britishcolumbia,canada,35.693353


In [307]:
users = users.drop('location', axis=1)

In [343]:
users = users.drop('cma', axis=1)

In [344]:
users

,user_id,age,location_state,location_country
0,8,35,ontario,canada
1,11400,49,ontario,canada
2,67544,30,ontario,canada
3,85526,36,britishcolumbia,canada
4,96054,29,ontario,canada
...,...,...,...,...
66785,278376,54,california,usa
66786,278621,74,delaware,canada
66787,278636,37,alabama,usa
66788,278659,33,britishcolumbia,canada


In [347]:
users.isnull().sum()

user_id             0
age                 0
location_state      0
location_country    0
dtype: int64

In [346]:
users.to_csv('./data/users_pppp.csv', index=False) # 가공된 users 내보내기 # 가장 최신 ppp2 은 id age state country 존재(any 안쓰고, 나이만 채움) 

In [260]:
userspp = pd.read_csv('./data/users_pp.csv')

In [263]:
userspp = userspp.drop('location_state', axis = 1)

In [265]:
userspp.to_csv('./data/users_pp2.csv', index=False)

# books

In [76]:
books # 149570권 중 웬만하면 삭제하고 싶지 않음 # 근데 68851개나 카테고리가 비어 있음

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['Humor'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Nature'],A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Fiction'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [77]:
books['year_of_publication'] = books['year_of_publication'].astype(int) # 출판년도를 정수로

In [78]:
books = books.drop('summary', axis=1) # summary 삭제

In [79]:
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip()) # 일단 category에서 대괄호 밖으로 빼기
books['category'] = books['category'].str.lower() # 소문자로 바꾸기

In [80]:
books['category_high'] = books['category'].copy() # category_high로 category를 복사

In [81]:
groupings = {'Fiction': ['fiction'], # 너무 넓으니 맨 위로 빼자
             'Literature & Poem': ['liter', 'poem', 'poetry'],
             'Science & Math': ['science', 'math', 'logy'], # science, logy 범위가 너무 넓으니 맨 위로
             'Parenting & Relationships': ['baby', 'parent', 'family', 'tionship', 'brother', 'sister'], # 좀 큼
             'Medical Books': ['medi', 'psycho'], # psy의 세분화 가능
             'Animal & Nature': ['animal', 'ecolo', 'plant', 'nature'],
             
             'Arts & Photography': ['art', 'photo'], # art는 겹치는 글자가 너무 많음
             'Biographies & Memoirs': ['biog', 'memo'],
             'Business & Money': ['busi', 'money', 'econo'],
             'Calendars': ['calen'],
             'Children\'s Books': ['child', 'baby'],
             'Christian Books & Bibles': ['christi', 'bible'], #크리스마스때매
             'Comics & Graphic Novels': ['comics', 'graphic novel'],
             'Computers & Technology': ['computer', 'techno', 'archi'],
             'Cookbooks, Food & Wine': ['cook'],
             'Crafts, Hobbies & Home': ['crafts'],
             'Education & Teaching': ['educa', 'teach'],
             'Engineering & Transportation': ['engine', 'transp'],
             'Health, Fitness & Dieting': ['health', 'fitness', 'diet'],
             'History': ['histo'],
             
             'Humor & Entertainment': ['humor', 'entertai', 'comed', 'game'],
             'Law': ['law'],
             'LGBTQ+ Books': ['lesbian', 'gay', 'bisex'],
             'Mystery, Thriller & Suspense': ['myste', 'thril', 'suspen'],
             'Politics & Social Sciences': ['politic', 'social'],
             'Reference': ['reference'],
             'Religion & Spirituality': ['religi'],
             'Romance': ['romance'],
             'Science Fiction & Fantasy': ['science fiction', 'fantasy'],
             'Self-Help': ['self'], # self 검색시 모두 자기계발 관련
             'Sports & Outdoors': ['exerc','sport','outdoor'],
             'Teen & Young Adult': ['teen', 'adol', 'juven'], #nonfiction이란 말은 청소년 관련뿐
             'Test Preparation': ['test', 'school', 'examina'],
             'Travel': ['travel'],
              }

In [82]:
for new_group, small in groupings.items(): # 파편화된 카테고리를 새 그룹으로 묶어주기
    for s in small:
        books.loc[books[books['category'].str.contains(s, na = False)].index, 'category_high'] = new_group

In [83]:
books

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,img_path,category_high
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,images/0002005018.01.THUMBZZZ.jpg,actresses
1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,1940 1949,images/0060973129.01.THUMBZZZ.jpg,1940 1949
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,medical,images/0374157065.01.THUMBZZZ.jpg,Medical Books
3,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,fiction,images/0399135782.01.THUMBZZZ.jpg,Fiction
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,history,images/0425176428.01.THUMBZZZ.jpg,History
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,humor,images/067161746X.01.THUMBZZZ.jpg,Humor & Entertainment
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,nature,images/0767907566.01.THUMBZZZ.jpg,Animal & Nature
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg,NaN
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,fiction,images/0912333022.01.THUMBZZZ.jpg,Fiction


In [84]:
books['category_high'] = books['category_high'].fillna('Unclassified') # 안 채워진 건 미분류 항목으로 넣기

In [85]:
books_count = books.groupby('category_high').count()['isbn'].to_dict() # category_high별 isbn 수?
for i in range(len(books)):
    books.at[i, 'count'] = books_count[books['category_high'][i]] # books에 count 항목이 생김 # 미분류가 68851개

In [96]:
for i in range(len(books)): # 5033개의 항목을 미분류로 편입
    if books.at[i, 'count'] < 10:
        books.at[i, 'category_high'] = 'Unclassified'

In [97]:
books_count = books.groupby('category_high').count()['isbn'].to_dict() # category_high별 isbn 수?
for i in range(len(books)):
    books.at[i, 'count'] = books_count[books['category_high'][i]] # 다시 세보자 미분류가 5033개 늘었으면 성공

In [98]:
books # 73884개의 미분류 나옴

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,img_path,category_high,count
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,actresses,images/0002005018.01.THUMBZZZ.jpg,actresses,12.0
1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,1940 1949,images/0060973129.01.THUMBZZZ.jpg,Unclassified,73884.0
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,medical,images/0374157065.01.THUMBZZZ.jpg,Medical Books,1016.0
3,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,fiction,images/0399135782.01.THUMBZZZ.jpg,Fiction,33672.0
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,history,images/0425176428.01.THUMBZZZ.jpg,History,1994.0
...,...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,humor,images/067161746X.01.THUMBZZZ.jpg,Humor & Entertainment,1598.0
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,nature,images/0767907566.01.THUMBZZZ.jpg,Animal & Nature,736.0
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg,Unclassified,73884.0
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,fiction,images/0912333022.01.THUMBZZZ.jpg,Fiction,33672.0


In [99]:
books['book_title'] = books['book_title'].apply(text_preprocessing_func) # 제목 깨진거 수정

In [100]:
books['publisher'] = books['publisher'].apply(text_preprocessing_func) # 출판사 깨진거 수정
publisher_dict=(books['publisher'].value_counts()).to_dict() # 그리고 숫자 세기
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count']) # 출판사별 숫자?

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False) # count 수대로 정렬

modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values # 1개 이상 있는 출판사의 경우
for publisher in modify_list:
    try:
        number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
        right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
        books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
    except: 
        pass

In [102]:
isbn_dict = {} # isbn으로 language를 채우자
isbn_dict = { books['language'][idx] : [isbn[:3]] if books['language'][idx] not in isbn_dict.keys() else isbn_dict[books['language'][idx]].append(isbn[:2]) for idx, isbn in enumerate(books['isbn'])}
isbn_code = {'0' : 'english', '1' : 'english', '2': 'franch', '3' : 'german', '4' : 'japan', '5' : 'russia', '7' : 'china',
             '65' : 'brazil', '80' : 'czecho', '81' : 'india', '82' : 'norway', '83' : 'poland', '84' : 'espanol', '85' : 'brazil', '86' : 'yugoslavia', '87' : 'danish', '88' : 'italy', '89' : 'korean', '90' : 'netherlands', '91' : 'sweden',
            '92' : 'international ngo', '93' : 'inida', '94' : 'netherlands', '600' : 'iran', '601' : 'kazakhstan', '602' : 'indonesia', '603' : 'saudi arabia', '604' : 'vietnam', '605' : 'turkey',
            '606' : 'romania', '607' : 'mexico', '608' : 'north macedonia', '609' : 'lithuania', '611' : 'thailand', '612' : 'peru', '613' : 'mauritius',
            '614' : 'lebanon', '615' : 'hungary', '616' : 'thailand', '617' : 'ukraine', '618' : 'greece', '619' : 'bulgaria', '620' : 'mauritius', '621' : 'phillippines',
            '622' : 'iran', '623' : 'indonesia', '624' : 'sri lanka', '625' : 'turkey', '626' : 'taiwan', '627' : 'pakistan', '628' : 'colombia', '629' : 'malaysia', '630' : 'romania',
            '950' : 'argentina', '951' : 'finland', '952' : 'finland', '953' : 'croatia', '954' : 'bulgaria', '955' : 'sri lanka',
            '956' : 'chile', '957' : 'taiwan', '958' : 'colombia', '959' : 'cuba', '960' : 'greece' , '961' : 'slovenia', '962' : 'hong kong',
            '963' : 'hungary', '964' : 'iran', '965' : 'israel', '966' : 'urkaine', '967' : 'malaysia', '968' : 'mexico', '969' : 'pakistan', '970' : 'mexico',
            '971' : 'phillippines', '972' : 'portugal', '973' : 'romania', '974' : 'thailand', '975' : 'turkey', '976' : 'caribbean community', '977' : 'egypt', '978' : 'nigeria', 
            '979' : 'indonesia', '980' : 'venezuela', '981' : 'singapore', '982' : 'south pacific', '983' : 'malaysia', '984' : 'bangladesh', '985' : 'velarus', '986' : 'taiwan',
            '987' : 'argentina', '988' : 'hong kong', '989' : 'portugal',
            '9960':'saudi arabia', '9963' : 'cyprus', '9968' : 'costa rica', '9971' : 'singapore', '9972' : 'peru', '9974' : 'uruguay',
            '9976' : 'tanzania', '9977' : 'costa rica', '9979' : 'iceland', '9986' : 'lithuania',
            '99903' : 'mauritius', '99905' : ' bolivia', '99909' : 'malta', '99912' : 'botswana', '99920' : 'andorra', '99928' : 'georgia',
            '99935' : 'haiti', '99936' : 'bhutan', '99942' : 'armenia', '99943' : 'albania', '99974' : 'bolivia',
            '99975' : 'mongolia', '99989' : 'paraguay'}
check_list = []
books['isbn_country'] = 'na'
for idx in range(len(books)):
    isbn = books['isbn'][idx][:5]
    if isbn[0] in isbn_code.keys():
        books.at[idx, 'isbn_country'] = isbn_code[isbn[0]]
    elif isbn[:2] in isbn_code.keys():
        books.at[idx, 'isbn_country'] = isbn_code[isbn[0:2]]
    elif isbn[:3] in isbn_code.keys():
        books.at[idx, 'isbn_country'] = isbn_code[isbn[0:3]]
    elif isbn[:4] in isbn_code.keys():
        books.at[idx, 'isbn_country'] = isbn_code[isbn[:4]]
    elif isbn[:] in isbn_code.keys():
        books.at[idx, 'isbn_country'] = isbn_code[isbn[:]]
    else:
        check_list.append(isbn)

books[books['isbn_country'] == 'na']['isbn_country'] = 'english' # nan이면 en으로 채움

/tmp/ipykernel_28851/1647494993.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books[books['isbn_country'] == 'na']['isbn_country'] = 'english'


In [104]:
books['book_author'] = books['book_author'].apply(text_preprocessing_func) # 작가 깨진거 수정

In [106]:
books = books.drop(['language'], axis = 1)
books = books.drop(['count'], axis = 1) # 필요없는 칼럼 삭제

In [108]:
books = books.drop(['category'], axis = 1) # 필요없는 칼럼 삭제

In [109]:
books

,isbn,book_title,book_author,year_of_publication,publisher,img_url,img_path,category_high,isbn_country
0,0002005018,clara callan,richard bruce wright,2001,harpercollins,http://images.amazon.com/images/P/0002005018.0...,images/0002005018.01.THUMBZZZ.jpg,actresses,english
1,0060973129,decision in normandy,carlo d'este,1991,harpercollins,http://images.amazon.com/images/P/0060973129.0...,images/0060973129.01.THUMBZZZ.jpg,Unclassified,english
2,0374157065,flu: the story of the great influenza pandemic...,gina bari kolata,1999,farrar straus giroux,http://images.amazon.com/images/P/0374157065.0...,images/0374157065.01.THUMBZZZ.jpg,Medical Books,english
3,0399135782,the kitchen god's wife,amy tan,1991,putnam pub group,http://images.amazon.com/images/P/0399135782.0...,images/0399135782.01.THUMBZZZ.jpg,Fiction,english
4,0425176428,what if?: the world's foremost military histor...,robert cowley,2000,berkley publishing group,http://images.amazon.com/images/P/0425176428.0...,images/0425176428.01.THUMBZZZ.jpg,History,english
...,...,...,...,...,...,...,...,...,...
149565,067161746X,the bachelor home companion: a practical guide...,p.j. o'rourke,1987,pocket,http://images.amazon.com/images/P/067161746X.0...,images/067161746X.01.THUMBZZZ.jpg,Humor & Entertainment,english
149566,0767907566,all elevations unknown: an adventure in the he...,sam lightner,2001,broadway books,http://images.amazon.com/images/P/0767907566.0...,images/0767907566.01.THUMBZZZ.jpg,Animal & Nature,english
149567,0884159221,why stop?: a guide to texas historical roadsid...,claude dooley,1985,bridge publications,http://images.amazon.com/images/P/0884159221.0...,images/0884159221.01.THUMBZZZ.jpg,Unclassified,english
149568,0912333022,the are you being served? stories: 'camping in...,jeremy lloyd,1997,pub group west,http://images.amazon.com/images/P/0912333022.0...,images/0912333022.01.THUMBZZZ.jpg,Fiction,english


In [110]:
books.to_csv('./books_pp.csv', index=False)

# user_pp (id, state, country) 보완
- age, state 관련 수많은 시도를 했지만 의미 없음

# books_pp(isbn, title, author, year, publisher, img, category_high, isbn_country) 보완
- category_high와 isbn_country는 일단 두자.
- title, author, year, publisher도 카테고리처럼 세부 조정 해보자

In [389]:
bookspp = pd.read_csv("./data/books_pp.csv", encoding='utf-8') # 불러오기

In [390]:
bookspp[bookspp['publisher'].str.contains('pengu')]['publisher'].unique() # array(['penguin books', 'penguin books ltd'], dtype=object) 이처럼 같은 penguin books 인데도 다른 이름

array(['penguin books', 'penguin books ltd'], dtype=object)

In [391]:
kk = bookspp.groupby('publisher')['isbn'].count().sort_values(ascending = False) # 출판사별 isbn 수 정렬

In [392]:
kk.head(10) # 상위 10개 출판사

publisher
pocket                7306
harlequin             5268
bantam books          4753
harpercollins         4518
st. martin's press    4043
vintage books usa     3822
penguin books         3807
ballantine books      3090
signet book           3068
avon                  2792
Name: isbn, dtype: int64

In [393]:
bookspp[bookspp['isbn'].apply(lambda x: x[:4])=='0140']['publisher'].unique() # isbn으로 정리한 거라 문제 없음

array(['penguin books'], dtype=object)

In [394]:
bookspp['publisher'].nunique() # 그럼 총 1514개의 출판사

1514

In [395]:
ff = bookspp.groupby('book_author')['isbn'].count().sort_values(ascending = False) # 작가별 isbn 수 정렬
ff

book_author
stephen king           453
agatha christie        407
william shakespeare    383
barbara cartland       284
nora roberts           245
                      ... 
iris, d. rainer          1
irma eskes               1
irma hildebrandt         1
irma kurtz               1
joseph vargo             1
Name: isbn, Length: 60221, dtype: int64

In [407]:
bookspp[bookspp['book_author'].str.contains('roberts')]['book_author'].unique()

array(['nora roberts', 'cokie roberts', 'shelly r. roberts',
       'rachel roberts', 'james c. robertson', 'robertson davies',
       'jane roberts', 'a.r.r.r. roberts', 'pat robertson',
       'willo davis roberts', 'monty roberts', 'mary roberts rinehart',
       'doris roberts', 'les roberts', 'roberts', 'john maddox roberts',
       'edgar v roberts', 'gillian roberts', 'laura peyton roberts',
       'lynn roberts', 'kenneth j. roberts', 'jane roberts wood',
       'victoria roberts', 'sheryl lindsell-roberts', 'garyn g. roberts',
       'yvonne roberts', 'david roberts', 'j.r. roberts', 'julia roberts',
       'charles robertson', 'kelsey roberts', 'ronald richard roberts',
       'timothy r. roberts', 'michele roberts', 'ruth aproberts',
       'e. arnot robertson', 'wess roberts', 'james l.  roberts',
       'j. l. roberts', 'janet louise roberts', 'elizabeth roberts',
       'james robertson', 'laurel robertson', 'adele roberts',
       'sarah roberts', 'robertson', 'ralph rob

# train_ppp(2개 평가한 애들 털어냄) 보완